In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import arviz as az
import pymc as pm
from pymc.math import dot, invlogit

In [2]:
df = pd.read_csv("wdbc.data", delimiter=",", header=None)
columns = ["id", "diagnosis", "radius1", "texture1", "perimeter1", "area1", "smoothness1", 
           "compactness1", "concavity1", "concave_points1", "symmetry1", "fractal_dimension1", 
           "radius2", "texture2", "perimeter2", "area2", "smoothness2", "compactness2", 
           "concavity2", "concave_points2", "symmetry2", "fractal_dimension2", "radius3", 
           "texture3", "perimeter3", "area3", "smoothness3", "compactness3", "concavity3", 
           "concave_points3", "symmetry3", "fractal_dimension3"]
df.columns = columns
df["diagnosis"] = (df["diagnosis"] == "M").astype("int32")
df.head(5)

,id,diagnosis,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
positive = df[df["diagnosis"] == 1]
negative = df[df["diagnosis"] == 0]
print(positive.shape, negative.shape)

(212, 32) (357, 32)


In [4]:
seed = 0
test_size = 200
test_pos_size = test_size // 2
test_neg_size = test_size - test_pos_size
test_pos_idx = np.random.choice(positive.shape[0], size=test_pos_size, replace=False)
test_neg_idx = np.random.choice(negative.shape[0], size=test_neg_size, replace=False)
test_pos_samples = positive.iloc[test_pos_idx]
test_neg_samples = negative.iloc[test_neg_idx]
test_subset = pd.concat([test_pos_samples, test_neg_samples])

train_positive = positive[~positive.index.isin(test_pos_samples.index)]
train_negative = negative[~negative.index.isin(test_neg_samples.index)]
train_subset = pd.concat([train_positive, train_negative])
print(train_subset.shape, test_subset.shape)
assert set(train_subset.index) & set(test_subset.index) == set()
assert len(set(train_subset.index)) == len(train_subset.index)
assert len(set(test_subset.index)) == len(test_subset.index)

(369, 32) (200, 32)


In [5]:
def balance_dataset(data, num_samples, positive_perc, seed):
    np.random.seed(seed)
    positive = data[data["diagnosis"] == 1]
    negative = data[data["diagnosis"] == 0]
    pos_num_samples = int(num_samples * positive_perc)
    neg_num_samples = num_samples - pos_num_samples
    pos_idx = np.random.choice(positive.shape[0], size=pos_num_samples, replace=False)
    neg_idx = np.random.choice(negative.shape[0], size=neg_num_samples, replace=False)
    pos_samples = positive.iloc[pos_idx]
    neg_samples = negative.iloc[neg_idx]
    subset = pd.concat([pos_samples, neg_samples])
    return subset

In [33]:
def compute_metrics(y_true, y_pred, verbose=True):
    eval_df = pd.DataFrame({"y_true": y_true, "y_pred": y_pred})
    tp = eval_df[(eval_df["y_true"] == 1) & (eval_df["y_pred"] == 1)].shape[0]
    fp = eval_df[(eval_df["y_true"] == 0) & (eval_df["y_pred"] == 1)].shape[0]
    fn = eval_df[(eval_df["y_true"] == 1) & (eval_df["y_pred"] == 0)].shape[0]
    tn = eval_df[(eval_df["y_true"] == 0) & (eval_df["y_pred"] == 0)].shape[0]
    pos = sum(y_true)
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall > 0) else 0
    if verbose:
        print(f"Positive Percentage: {pos / len(y_true)}")
        print(f"True Positives: {tp}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Negatives: {tn}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    return metrics

In [ ]:
def bayes_modeling(sigma, X_train, y_train, X_test_imb, y_test_imb, X_test_bal, y_test_bal, seed):
    results = {}
    print(f"Training Bayesian logistic regression with sigma = {sigma}, {X_train.shape[0]} training samples, {100 * y_train.mean()}% positive...")
    with pm.Model() as model:
        X_data = pm.Data("X_data", X_train)
        y_data = pm.Data("y_data", y_train)

        intercept = pm.Normal("intercept", mu=0, sigma=10)
        beta = pm.Normal("beta", mu=0, sigma=sigma, shape=X_train.shape[1])
        p = invlogit(intercept + dot(X_data, beta))
        pm.Bernoulli("y", p=p, observed=y_data, shape=X_data.shape[0])
        trace = pm.sample(3000, random_seed=seed)
        results["trace"] = trace
    print()
    print("Evaluating on training set...")
    with model:
        ppc_train_imb = pm.sample_posterior_predictive(trace, random_seed=seed)
    y_train_prob = ppc_train_imb.posterior_predictive.mean(
        dim=["chain", "draw"]
    ).y.values
    y_train_pred = (y_train_prob >= 0.5) * 1
    train_metrics = compute_metrics(y_train, y_train_pred)
    results["train"] = train_metrics
    print()

    print(f"Evaluating on imbalanced testing set, {X_test_imb.shape[0]} testing samples, {100 * y_test_imb.mean()}% positive...")
    with model:
        pm.set_data({"X_data": X_test_imb})
        ppc_test_imb = pm.sample_posterior_predictive(trace, random_seed=seed)
    y_test_imb_prob = ppc_test_imb.posterior_predictive.mean(
        dim=["chain", "draw"]
    ).y.values
    y_test_imb_pred = (y_test_imb_prob >= 0.5) * 1
    test_imb_metrics = compute_metrics(y_test_imb, y_test_imb_pred)
    results["test_imb"] = test_imb_metrics
    print()

    print(f"Evaluating on balanced testing set, {X_test_bal.shape[0]} testing samples, {100 * y_test_bal.mean()}% positive...")
    with model:
        pm.set_data({"X_data": X_test_bal})
        ppc_test_bal = pm.sample_posterior_predictive(trace, random_seed=seed)
    y_test_bal_prob = ppc_test_bal.posterior_predictive.mean(
        dim=["chain", "draw"]
    ).y.values
    y_test_bal_pred = (y_test_bal_prob >= 0.5) * 1
    test_bal_metrics = compute_metrics(y_test_bal, y_test_bal_pred)
    results["test_bal"] = test_bal_metrics
    print()
    return results

In [62]:
seed = 5
train_num_samples = 150
train_imb = balance_dataset(train_subset, train_num_samples, 0.2, seed)
X_train_imb = train_imb.drop(["id", "diagnosis"], axis=1)
y_train_imb = train_imb["diagnosis"]
print(X_train_imb.shape, y_train_imb.shape)
print(f"Positive class percentage {y_train_imb.mean()}")

scaler = StandardScaler()
scaler.fit(X_train_imb)
X_train_imb_scaled = scaler.transform(X_train_imb)

test_num_samples = 125
test_imb = balance_dataset(test_subset, test_num_samples, 0.2, seed)
X_test_imb = test_imb.drop(["id", "diagnosis"], axis=1)
y_test_imb = test_imb["diagnosis"]
print(X_test_imb.shape, y_test_imb.shape)
print(f"Positive class percentage {y_test_imb.mean()}")
X_test_imb_scaled = scaler.transform(X_test_imb)

test_num_samples = 150
test_bal = balance_dataset(test_subset, test_num_samples, 0.5, seed)
X_test_bal = test_bal.drop(["id", "diagnosis"], axis=1)
y_test_bal = test_bal["diagnosis"]
print(X_test_bal.shape, y_test_bal.shape)
print(f"Positive class percentage {y_test_bal.mean()}")
X_test_bal_scaled = scaler.transform(X_test_bal)

(150, 30) (150,)
Positive class percentage 0.2
(125, 30) (125,)
Positive class percentage 0.2
(150, 30) (150,)
Positive class percentage 0.5


In [65]:
sigmas = [0.1, 1, 5]
results = {}
seed = 5
for sigma in sigmas:
    result = bayes_modeling(sigma=sigma, X_train=X_train_imb_scaled, y_train=y_train_imb, X_test_imb=X_test_imb_scaled, y_test_imb=y_test_imb, 
                            X_test_bal=X_test_bal_scaled, y_test_bal=y_test_bal, seed=seed)
    results[sigma] = result

Training Bayesian logistic regression with sigma = 0.1, 150 training samples, 20.0% positive...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [intercept, beta]


Output()

Sampling 2 chains for 1_000 tune and 3_000 draw iterations (2_000 + 6_000 draws total) took 33 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [y]


Output()

...

Positive Percentage: 0.2
True Positives: 30
False Positives: 0
False Negatives: 0
True Negatives: 120
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0

Evaluating on imbalanced testing set, 125 testing samples, 20.0% positive...


Sampling: [y]


Output()

Positive Percentage: 0.2
True Positives: 22
False Positives: 0
False Negatives: 3
True Negatives: 100
Accuracy: 0.976
Precision: 1.0
Recall: 0.88
F1 score: 0.9361702127659575

Evaluating on balanced testing set, 150 testing samples, 50.0% positive...


Positive Percentage: 0.5
True Positives: 69
False Positives: 0
False Negatives: 6
True Negatives: 75
Accuracy: 0.96
Precision: 1.0
Recall: 0.92
F1 score: 0.9583333333333334



In [68]:
for k,v in results.items():
    print(k, v["train"])

0.1 {'accuracy': 0.92, 'precision': 1.0, 'recall': 0.6, 'f1': 0.7499999999999999}
1 {'accuracy': 0.9933333333333333, 'precision': 1.0, 'recall': 0.9666666666666667, 'f1': 0.983050847457627}
5 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


In [67]:
for k,v in results.items():
    print(k, v["test_imb"])

0.1 {'accuracy': 0.944, 'precision': 1.0, 'recall': 0.72, 'f1': 0.8372093023255813}
1 {'accuracy': 0.976, 'precision': 1.0, 'recall': 0.88, 'f1': 0.9361702127659575}
5 {'accuracy': 0.976, 'precision': 1.0, 'recall': 0.88, 'f1': 0.9361702127659575}


In [66]:
for k,v in results.items():
    print(k, v["test_bal"])

0.1 {'accuracy': 0.8466666666666667, 'precision': 1.0, 'recall': 0.6933333333333334, 'f1': 0.8188976377952756}
1 {'accuracy': 0.9666666666666667, 'precision': 1.0, 'recall': 0.9333333333333333, 'f1': 0.9655172413793104}
5 {'accuracy': 0.96, 'precision': 1.0, 'recall': 0.92, 'f1': 0.9583333333333334}


In [110]:
az.summary(results[0.1]["trace"], hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-1.813,0.262,-2.342,-1.325,0.002,0.004,12826.0,4416.0,1.0
beta[0],0.103,0.098,-0.080,0.305,0.001,0.002,11874.0,4090.0,1.0
beta[1],0.083,0.095,-0.107,0.268,0.001,0.001,15119.0,4180.0,1.0
beta[2],0.105,0.100,-0.084,0.304,0.001,0.001,12011.0,4338.0,1.0
beta[3],0.100,0.095,-0.076,0.301,0.001,0.001,14181.0,4207.0,1.0
beta[4],0.051,0.094,-0.132,0.233,0.001,0.001,13424.0,4301.0,1.0
beta[5],0.060,0.097,-0.130,0.240,0.001,0.001,13518.0,4680.0,1.0
beta[6],0.098,0.096,-0.091,0.282,0.001,0.002,12928.0,4626.0,1.0
beta[7],0.117,0.097,-0.078,0.300,0.001,0.002,14646.0,4360.0,1.0
beta[8],0.031,0.095,-0.149,0.220,0.001,0.001,17779.0,4732.0,1.0


In [111]:
az.summary(results[1]["trace"], hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-4.919,1.002,-6.822,-2.976,0.016,0.013,4283.0,4123.0,1.0
beta[0],0.244,0.935,-1.567,2.041,0.011,0.013,7416.0,4445.0,1.0
beta[1],0.295,0.740,-1.071,1.823,0.011,0.009,4698.0,4340.0,1.0
beta[2],0.303,0.926,-1.477,2.141,0.011,0.013,7445.0,4199.0,1.0
beta[3],0.365,0.949,-1.515,2.214,0.010,0.015,9148.0,4287.0,1.0
beta[4],0.667,0.778,-0.914,2.161,0.010,0.010,6164.0,4940.0,1.0
beta[5],0.072,0.880,-1.636,1.792,0.011,0.011,6586.0,4612.0,1.0
beta[6],0.869,0.887,-0.871,2.583,0.011,0.011,6147.0,4590.0,1.0
beta[7],0.825,0.877,-0.855,2.561,0.010,0.011,7663.0,4831.0,1.0
beta[8],-0.248,0.738,-1.672,1.171,0.010,0.009,5227.0,4045.0,1.0


In [112]:
az.summary(results[5]["trace"], hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-16.182,3.803,-23.985,-9.375,0.063,0.048,3785.0,3572.0,1.0
beta[0],0.378,4.503,-8.291,8.980,0.055,0.061,6778.0,4173.0,1.0
beta[1],-0.708,3.111,-6.561,5.670,0.049,0.038,4019.0,3942.0,1.0
beta[2],0.656,4.601,-8.347,9.783,0.050,0.068,8406.0,4405.0,1.0
beta[3],1.414,4.541,-7.477,10.146,0.050,0.064,8258.0,4198.0,1.0
beta[4],2.799,3.450,-3.478,10.032,0.049,0.046,4868.0,4115.0,1.0
beta[5],-0.047,4.168,-8.215,7.812,0.052,0.054,6502.0,4629.0,1.0
beta[6],5.715,4.060,-1.889,13.818,0.049,0.054,6926.0,4437.0,1.0
beta[7],4.289,4.070,-3.760,12.144,0.048,0.058,7304.0,4294.0,1.0
beta[8],-1.400,3.022,-7.363,4.259,0.041,0.039,5433.0,4329.0,1.0
